In [2]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
from os.path import expanduser


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('iphonecamerareviews.csv')
df.head(5)

,stars,comment
0,5,The iPhone design is good and the camera quali...
1,5,"This being my second Apple device. No, I haven..."
2,5,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...
3,5,I bought the iPhone 11 64gb red and I absolute...
4,5,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    910 non-null    int64 
 1   comment  910 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.3+ KB


In [6]:
df.describe().round(1)

,stars
count,910.0
mean,4.7
std,0.6
min,3.0
25%,5.0
50%,5.0
75%,5.0
max,5.0


In [7]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [8]:
df= df.dropna()
df.shape

(910, 2)

In [9]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,The iPhone design is good and the camera quali...,56
1,"This being my second Apple device. No, I haven...",152
2,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...,19
3,I bought the iPhone 11 64gb red and I absolute...,74
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0


In [11]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [12]:
df[['comment','punctuation']].head()

,comment,punctuation
0,The iPhone design is good and the camera quali...,33
1,"This being my second Apple device. No, I haven...",61
2,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...,4
3,I bought the iPhone 11 64gb red and I absolute...,20
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,1


In [13]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,The iPhone design is good and the camera quali...,0
1,"This being my second Apple device. No, I haven...",0
2,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...,0
3,I bought the iPhone 11 64gb red and I absolute...,0
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0


In [14]:
df.hastags.loc[df.hastags != 0].count()

0

In [15]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,The iPhone design is good and the camera quali...,0
1,"This being my second Apple device. No, I haven...",4
2,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...,0
3,I bought the iPhone 11 64gb red and I absolute...,1
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0


In [16]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,The iPhone design is good and the camera quali...,10
1,"This being my second Apple device. No, I haven...",10
2,\n\n\n\n\n\n\n\n\n\n \n \n \n This phone...,0
3,I bought the iPhone 11 64gb red and I absolute...,5
4,\n\n\n\n\n\n\n\n\n\n \n \n \n Awesome ph...,0


In [17]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    the iphone design is good and the camera quali...
1    this being my second apple device. no, i haven...
2    this phone is bit big to hold but good for and...
3    i bought the iphone 11 64gb red and i absolute...
4                 awesome phone great camera quality .
Name: comment, dtype: object

In [18]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-18-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    the iphone design is good and the camera quali...
1    this being my second apple device no i havent ...
2    this phone is bit big to hold but good for and...
3    i bought the iphone 11 64gb red and i absolute...
4                  awesome phone great camera quality 
Name: comment, dtype: object

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

352    phone bit big hold good android usersas per de...
158    last 3 months waiting best possible phone ever...
37     proscamera quality 0910display 0910battery lif...
385    iphone 11 good phone every thing fine budget p...
103    bought iphone 11 64gb red absolutely love desi...
96     almost 10 months using iphone 11 128 gb apple ...
436    almost 10 months using iphone 11 128 gb apple ...
539    3 years iphone 7 finally decided take upgrade ...
826    almost 10 months using iphone 11 128 gb apple ...
556    almost 10 months using iphone 11 128 gb apple ...
Name: comment, dtype: object

In [20]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [21]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [22]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [23]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [24]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [25]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [26]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [27]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [28]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [29]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [30]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [31]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    phone design good camera quality awesome also ...
1    second apple device haven sold kidney got ask ...
2    phone bit big hold good andros users per desig...
3    bought phone 11 64gb red absolutely love desig...
4                   awesome phone great camera quality
Name: comment, dtype: object

In [32]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [33]:
df['comment'] = df.comment.apply(round1)
df.comment

0      iphone design good camera quality awesome also...
1      second apple device havent sold kidney got  av...
2      phone bit big hold good android usersas per de...
3      bought iphone   red absolutely love design ins...
4                     awesome phone great camera quality
                             ...                        
905    iphone  good phone every thing fine budget pho...
906    almost  months using iphone   gb apple known o...
907    proscamera quality   life  issue face heating ...
908    last  months waiting best possible phone every...
909     years iphone  finally decided take upgrade ip...
Name: comment, Length: 910, dtype: object

In [34]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [35]:
df['comment'] = df.comment.apply(round2)
df.comment

0      iphone design good camera quality awesome also...
1      second apple device havent sold kidney got  av...
2      phone bit big hold good android usersas per de...
3      bought iphone   red absolutely love design ins...
4                     awesome phone great camera quality
                             ...                        
905    iphone  good phone every thing fine budget pho...
906    almost  months using iphone   gb apple known o...
907    proscamera quality   life  issue face heating ...
908    last  months waiting best possible phone every...
909     years iphone  finally decided take upgrade ip...
Name: comment, Length: 910, dtype: object

In [36]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

phone      2912
iphone     2002
apple      1274
good       1183
quality    1092
screen     1001
using      1001
phones      819
camera      728
face        637
say         637
lcd         546
one         546
amazing     546
charger     546
like        546
ios         546
got         546
android     455
thing       455
dtype: int64

In [37]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,iphone design good camera quality awesome also...,97
1,second apple device havent sold kidney got av...,223
2,phone bit big hold good android usersas per de...,29
3,bought iphone red absolutely love design ins...,85
4,awesome phone great camera quality,5


In [38]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [39]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,iphone design good camera quality awesome also...,591
1,second apple device havent sold kidney got av...,1429
2,phone bit big hold good android usersas per de...,181
3,bought iphone red absolutely love design ins...,568
4,awesome phone great camera quality,34


In [40]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [41]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,iphone design good camera quality awesome also...,5.8
1,second apple device havent sold kidney got av...,5.7
2,phone bit big hold good android usersas per de...,5.3
3,bought iphone red absolutely love design ins...,5.9
4,awesome phone great camera quality,6.0


In [42]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
409,5,years iphone finally decided take upgrade ip...,30,11,0,3,2,47,276,5.6
699,5,years iphone finally decided take upgrade ip...,30,11,0,3,2,47,276,5.6


In [43]:
#df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [44]:
#df= df.drop('ProfileName', axis= 1)

In [45]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [46]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
806,4,almost months using iphone gb apple known o...,282,92,0,9,33,382,2635,6.2
348,5,last months waiting best possible phone every...,143,31,0,7,3,178,1085,5.5
814,5,awesome phone great camera quality,0,1,0,0,0,5,34,6.0
65,3,iphone good phone every thing fine budget pho...,54,6,0,4,0,61,343,5.0
601,5,second apple device havent sold kidney got av...,152,61,0,4,10,223,1429,5.7


In [47]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes"      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [48]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

735    iphone good phone every thing fine budget phon...
336    almost months using iphone gb apple known one ...
505    iphone good phone every thing fine budget phon...
305    iphone good phone every thing fine budget phon...
347    proscamera quality life issue face heating iss...
867    proscamera quality life issue face heating iss...
217    proscamera quality life issue face heating iss...
180    iphone design good camera quality awesome also...
129    years iphone finally decided take upgrade ipho...
3      bought iphone red absolutely love design insan...
Name: comment, dtype: object

In [49]:
df1= df

In [50]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [51]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [52]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0      iphone design good camera quality awesome also...
1      second apple device havent sold kidney got avi...
2      phone bit big hold good android usersas per de...
3      bought iphone red absolutely love design insan...
4                       awesome phone great camera quali
                             ...                        
905    iphone good phone every thing fine budget phon...
906    almost months using iphone gb apple known one ...
907    proscamera quality life issue face heating iss...
908    last months waiting best possible phone every ...
909    years iphone finally decided take upgrade ipho...
Name: comment, Length: 910, dtype: object

In [53]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [54]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [55]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [56]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

phone         2912
iphone        2002
apple         1274
good          1183
using         1001
quality       1001
screen        1001
phones         819
camera         728
face           637
say            637
amazing        546
one            546
got            546
lcd            546
ios            546
charger        546
great          455
android        455
thing          455
oled           455
rate           455
know           455
better         455
display        364
years          364
update         364
build          364
must           364
products       364
last           364
box            364
premium        364
every          364
review         364
video          364
might          364
life           364
dont           364
im             364
refresh        364
device         364
yes            364
works          364
battery        364
wont           364
charge         364
design         364
software       273
absolutely     273
dtype: int64

In [57]:
df.to_csv('iphonecamerareviews2.csv', index=False)